In [22]:
import pandas as pd
df = pd.read_json('metadata/Meta_v4.json')

In [23]:
UKS = []

for i in range(len(df)):
    for v, k in enumerate(df.loc[i, 'specifications']):
        if k not in UKS:
            UKS.append(k)
            
print(len(UKS))

102


In [24]:
def extract_unit(query, sen):
    start = re.search(query.lower(), sen.lower()).start()
    value = sen[0:start]
    num = "\d\d*(\.\d)?\d?"
    value = re.search(num.lower(), value.lower()).group(0)
    unit = re.search(query.lower(), sen.lower()).group(0)
    return value, unit

In [25]:
import re

def extract_unit(query, sen):
    start = re.search(query.lower(), sen.lower()).start()
    value = sen[0:start].strip()
    unit = re.search(query.lower(), sen.lower()).group(0)
    return value, unit

match = [r"ghz", r"mhz", r"gb", r"kb", r"tb", r"mb", r"mp", r"db", r"lbs", r"pounds", r"ounces", r"watt", r"volt", r"inches", "\""]
unit_list = ["camera_resolution", "hdd", "ssd", "Hard Drive", "RAM", "memory", "Memory Speed", "Graphics Card Ram Size", "weight", "Flash Memory Size", "Wattage", "speed", "Graphic Card Ram Size", "Size", "Display Size", "Chassis Size", "Line Size", "Sheet Size", "Width (inches)", "Item Dimensions", "Voltage"]

for j in match:
    for i in range(len(df)):
        for ind, k in enumerate(df.loc[i, 'specifications']):
            val = df.loc[i, 'specifications'][k]
            if (type(val) == tuple):
                continue                
            if ((k in unit_list) and (j in val.lower())):
                value, unit = extract_unit(j, val)
                df.loc[i, 'specifications'][k] = (value, unit, val)
                continue
                    
match = [r"ma", r"mbps", r"gbps", r"mb/s", r"gb/s"]
unit_list = ["battery", "transfer_speed"]
for j in match:
    for i in range(len(df)):
        for ind, k in enumerate(df.loc[i, 'specifications']):
            val = df.loc[i, 'specifications'][k]
            if (type(val) == tuple):
                continue                
            if ((k in unit_list) and (j in val.lower())):
                value, unit = extract_unit(j, val)
                df.loc[i, 'specifications'][k] = (value, unit, val)
                continue
                    
def handle_screen(reso):
    reso = reso.lower()
    new = re.sub('x', '*', reso)
    new = re.sub(' ', '', new)
    return new

for i in range(len(df)):
    try:
        df.loc[i, "specifications"]["screen_resolution"] = handle_screen(df.loc[i, "specifications"]["screen_resolution"])
    except: continue
        
for i in range(len(df)):
    try:
        if df.loc[i]['specifications']['Size'][1] == '"':
            df.loc[i]['specifications']['Size'] = list(df.loc[i]['specifications']['Size'])
            df.loc[i]['specifications']['Size'][1] = 'inches'
            df.loc[i]['specifications']['Size'] = tuple(df.loc[i]['specifications']['Size'])
            continue
        else: continue
    except: continue

In [26]:
df[~(df['specifications']=={})]['specifications']

0                            {'waterproof_rating': 'IPX7'}
1        {'Item Dimensions': ('6 x 4 x 0.1', 'inches', ...
100      {'screen_resolution': '1920*1080', 'camera_res...
1000     {'Item Package Quantity': '1', 'Number Of Piec...
10000    {'Item Dimensions': ('4.1 x 2.7 x 0.6', 'inche...
10003                               {'sensitivity': ' DB'}
10004                    {'Size': ('15', 'inches', '15"')}
10008    {'Item Dimensions': ('17.9 x 1.2 x 6.7', 'inch...
1001     {'transfer_speed': ('80', 'mb/s', ' 80 MB/s'),...
10011                    {'Size': ('11', 'inches', '11"')}
10014                      {'Size': ('4', 'inches', '4"')}
10015    {'memory': ('16', 'gb', '16gb'), 'Size': ('24'...
10020    {'screen_resolution': '2592*1944', 'Size': ('2...
10021    {'Item Dimensions': ('8.08 x 5.72 x 0.71', 'in...
10023    {'transfer_speed': ('80', 'mb/s', ' 80 MB/s'),...
10025                   {'screen_resolution': '1920*1080'}
10026    {'Item Dimensions': ('4.7 x 4.7 x 1', 'inches'.

In [27]:
cols_w_units = ["camera_resolution", "hdd", "ssd", "Hard Drive", "RAM", "memory", "Memory Speed\
", "weight", "Flash Memory Size", "Wattage", "speed", "Size", "Display Size", "transfer_speed", "battery"]

for j in cols_w_units:
    lst = []
    for i in range(len(df)):
        lst.append({})
    df[j] = lst
    
for i in range(len(df)):
    for j in cols_w_units:
        try:
            if (type(df.loc[i, "specifications"][j])==tuple):
                df.loc[i, j]['value'] = df.loc[i, "specifications"][j][0]
                df.loc[i, j]['unit'] = df.loc[i, "specifications"][j][1]
                
        except: continue

In [28]:
ctr = 0

df1 = df.iloc[:, -15:]

for i in range(len(df1)):
    for j in df1.columns:
        if not(df1.loc[i,j]=={}):
            df1.loc[i,j]['value'] = df1.loc[i,j]['value'].strip()
            
for i in range(len(df1)):
    for j in df1.columns:
        if not(df1.loc[i,j]=={}):
            try:
                df1.loc[i,j]['value'] = float(df1.loc[i,j]['value'])
            except:
                #print(df1.loc[i,j]['value'])
                
                lst = ''
                for k in range(len(df1.loc[i,j]['value'])):
                    if ((df1.loc[i,j]['value'][-k-1].isdigit()) or (df1.loc[i,j]['value'][-k-1]=='.') or (df1.loc[i,j]['value'][-k-1]==' ')):
                        lst = lst + df1.loc[i,j]['value'][-k-1]
                    else: break
                lst = lst.strip()
                try:
                    df1.loc[i,j]['value'] = float(lst.strip())
                except:
                    ctr+=1
                    del df1.loc[i,j]['value']
                    del df1.loc[i,j]['unit']

for i in range(len(df)):
    for j in df1.columns:
        if not(df1.loc[i,j]=={}):
            if ((not (df1.loc[i,j]['unit'] == 'gbps')) and (not (df1.loc[i,j]['unit'] == 'mbps'))):
                df1.loc[i,j]['unit'] = df1.loc[i,j]['unit'].upper()
                
df.iloc[:,-15:] = df1

In [35]:
atul = pd.read_csv('F:/AP/Dataset.csv')
df1 = atul[['asin', 'embedding_vectors']]
df1 = df1.merge(df, how = 'inner', on='asin')
df1.shape

(5236, 30)

In [38]:
df = df1.iloc[:, -15:]
for j in df.columns:
    units = []
    for i in range(len(df)):
        if j=='Size':
            if not(df.loc[i,j]=={}):
                if df.loc[i,j]['unit']=='GB':
                    try:
                        del df.loc[i,j]['unit']
                        del df.loc[i,j]['value']
                    except:
                        pass
        if j=='RAM':
            if not(df.loc[i,j]=={}):
                try:
                    if df.loc[i,j]['unit']=='TB':
                        try:
                            del df.loc[i,j]['unit']
                            del df.loc[i,j]['value']
                        except:
                            pass
                except:
                    del df.loc[i,j]['value']

In [39]:
df

,camera_resolution,hdd,ssd,Hard Drive,RAM,memory,Memory Speed,weight,Flash Memory Size,Wattage,speed,Size,Display Size,transfer_speed,battery
0,{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}
1,{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}
2,"{'value': 2.0, 'unit': 'MP'}",{},{},{},{},{},{},{},{},{},{},{},{},{},{}
3,{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}
4,{},{},{},{},{},{},{},{},{},{},{},"{'value': 15.0, 'unit': 'INCHES'}",{},{},{}
5,{},{},{},{},{},{},{},"{'value': 110.0, 'unit': 'LBS'}",{},{},{},"{'value': 19.0, 'unit': 'INCHES'}",{},{},{}
6,{},{},{},{},{},{},{},{},{},{},{},"{'value': 11.0, 'unit': 'INCHES'}",{},{},{}
7,{},{},{},{},{},{},{},{},{},{},{},"{'value': 4.0, 'unit': 'INCHES'}",{},{},{}
8,{},{},{},{},{},"{'value': 32.0, 'unit': 'GB'}",{},{},{},{},{},{},{},"{'value': 80.0, 'unit': 'MB/S'}",{}
9,{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}


In [40]:
df1.iloc[:,-15:] = df
df1.to_json('metadata/prod_final.json')

In [42]:
df = df1.iloc[:,-15:]
meta = pd.DataFrame(index=df.columns, columns=[0])
for j in df.columns:
    units = []
    for i in range(len(df)):   
        if not(df.loc[i,j]=={}):
            if (df.loc[i,j]['unit'] not in units):
                units.append(df.loc[i,j]['unit'])
    meta.loc[j,'0'] = units
    
meta = meta.iloc[:,-1]
print(meta)

meta.to_json('metadata/Product_units.json')

camera_resolution                          MP
hdd                                  [TB, GB]
ssd                                  [GB, TB]
Hard Drive                           [TB, GB]
RAM                              [GB, MB, KB]
memory                               [GB, TB]
Memory Speed                       [MHZ, GHZ]
weight                  [LBS, POUNDS, OUNCES]
Flash Memory Size                    [GB, TB]
Wattage                                [WATT]
speed                              [GHZ, MHZ]
Size                                 [INCHES]
Display Size                         [INCHES]
transfer_speed       [MB/S, gbps, mbps, GB/S]
battery                                  [MA]
Name: 0, dtype: object


In [43]:
df = df1
cats = []
for i in range(len(df)):
    for j in df.loc[i, 'category']:
        if j not in cats:
            cats.append(j)
UKS = []            
for i in range(len(df)):
    for ind, k in enumerate(df.loc[i, "specifications"]):
        if k not in UKS:
            UKS.append(k)
            
cats1 = []
for i in cats:
    i = i.replace('&amp;','&')
    cats1.append(i)
    
meta = pd.DataFrame(index = cats1, columns = UKS)

In [44]:
meta = meta.fillna(0)
for i in cats1:
    for ind in range(len(df)):
        if i in df.loc[ind, "category"]:
            for j, k in enumerate(df.loc[ind, "specifications"]):
                if k in UKS:
                    meta.loc[i, k] = meta.loc[i, k]+1

In [47]:
meta.to_json('metadata/Product_meta.json', orient = 'records')

In [49]:
meta[['Hard Drive', 'hdd', 'ssd']]

,Hard Drive,hdd,ssd
Electronics,359,92,87
Headphones,0,0,0
Earbud Headphones,0,0,0
Accessories & Supplies,0,12,0
Office Electronics Accessories,0,0,0
Camera & Photo,0,78,0
Video Surveillance,0,38,0
Surveillance Cameras,0,0,0
Bullet Cameras,0,0,0
Video,0,0,0
